Start of COlab

In [ ]:
!wget -O dtrb.zip -q https://github.com/lohzhunyewcs/deep-text-recognition-benchmark/archive/refs/heads/transformer.zip

In [ ]:
!unzip -q dtrb.zip

In [ ]:
import sys
sys.path.append('/content/deep-text-recognition-benchmark-transformer')

In [ ]:
!mkdir deep-text-recognition-benchmark-master/data

In [ ]:
!wget -O data_lmdb_release.zip https://www.dropbox.com/sh/i39abvnefllx2si/AABX4yjNn2iLeKZh1OAwJUffa/data_lmdb_release.zip?dl=1

In [ ]:
!unzip data_lmdb_release.zip

In [ ]:
!mv data_lmdb_release deep-text-recognition-benchmark-transformer/data/

In [ ]:
%cd deep-text-recognition-benchmark-transformer

In [ ]:
!pip install lmdb

In [ ]:
!python train.py \
--train_data data/training --valid_data data/validation \
--select_data MJ-ST --batch_ratio 0.5-0.5 \
--total_data_usage_ratio 0.001 \
--valInterval 1 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction TransformerDecoder

End of Colab

In [1]:
from model import load_model
from global_args import get_cfgs
import torch

In [2]:
args = ["--Transformation", "TPS", "--FeatureExtraction", "ResNet", "--SequenceModeling", "BiLSTM", "--Prediction", "TransformerDecoder"]
opt = get_cfgs(args)
model, converter = load_model(opt)

model input parameters 32 100 20 1 512 256 38 100 TPS ResNet BiLSTM TransformerDecoder
Skip Transformation.LocalizationNetwork.localization_fc2.weight as it is already initialized
Skip Transformation.LocalizationNetwork.localization_fc2.bias as it is already initialized
Model:
DataParallel(
  (module): Model(
    (Transformation): TPS_SpatialTransformerNetwork(
      (LocalizationNetwork): LocalizationNetwork(
        (conv): Sequential(
          (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (6): ReLU(inplace=True)
          (7): MaxPool2

In [3]:
model.module.Prediction

TransformerDecoder(
  (position_embeddings): PositionalEmbedding(
    (embeddings): Embedding(102, 256)
  )
  (word_embeddings): Embedding(102, 256)
  (layers): ModuleList(
    (0-5): 6 x TransformerDecoderLayer(
      (attention_1): AddNorm(
        (sublayer): MultiHeadAttention(
          (heads): ModuleList(
            (0-7): 8 x AttentionHead(
              (q): Linear(in_features=256, out_features=32, bias=True)
              (k): Linear(in_features=256, out_features=32, bias=True)
              (v): Linear(in_features=256, out_features=32, bias=True)
            )
          )
          (linear): Linear(in_features=256, out_features=256, bias=True)
        )
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (attention_2): AddNorm(
        (sublayer): MultiHeadAttention(
          (heads): ModuleList(
            (0-7): 8 x AttentionHead(
              (q): Linear(in_features=256, out_features=32, 

In [4]:
torch_inp = torch.rand(1, 1, 32, 100)
# text= torch.Tensor([1,2,3,4]).unsqueeze(0).long()
text, length = converter.encode(["abc"], batch_max_length=opt.batch_max_length)
print(f'{text.shape = }')
text = text[:, :-1]
output = model(torch_inp, text)

text.shape = torch.Size([1, 102])


In [5]:
model(torch_inp, text[:15]).shape

torch.Size([1, 101, 38])

In [ ]:
torch.Tensor([[0] for _ in range(5)]).shape

torch.Size([5, 1])

In [6]:
def generate_square_subsequent_mask(sz: int) -> torch.Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [44]:
ninety = torch.tril(torch.ones(5, 5))
ninety[ninety == 0] = -float('inf')
ninety

tensor([[1., -inf, -inf, -inf, -inf],
        [1., 1., -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1.]])

In [45]:
torch.ones((5,5)) * ninety

tensor([[1., -inf, -inf, -inf, -inf],
        [1., 1., -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1.]])

In [37]:
torch.triu(torch.ones(5, 5) * float('-inf'), diagonal=1)

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

In [27]:
mask = (torch.triu(torch.ones(5, 5) * float('-inf'), diagonal=1) == 0).int()
mask

tensor([[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1]], dtype=torch.int32)

In [31]:
inf_mask = ( torch.ones(5,5)  * float('-inf') ).masked_fill(mask=mask == 1, value=1)
inf_mask

tensor([[1., -inf, -inf, -inf, -inf],
        [1., 1., -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1.]])

In [33]:
torch.ones(5 ,5) * inf_mask

tensor([[1., -inf, -inf, -inf, -inf],
        [1., 1., -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1.]])

In [8]:
!python train.py \
--train_data data_lmdb_release/training --valid_data data_lmdb_release/validation \
--select_data MJ-ST --batch_ratio 0.5-0.5 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction TransformerDecoder

Traceback (most recent call last):
  File "c:\Users\ZY\Desktop\github_repos\lohzhunyewcs\deep-text-recognition-benchmark\train.py", line 16, in <module>
    from dataset import hierarchical_dataset, AlignCollate, Batch_Balanced_Dataset
  File "c:\Users\ZY\Desktop\github_repos\lohzhunyewcs\deep-text-recognition-benchmark\dataset.py", line 6, in <module>
    import lmdb
ModuleNotFoundError: No module named 'lmdb'
